In [5]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar']
instruments_aug = ['Accordion', 'Violin', 'Piano']

# from setup_logging import setup_logging
# setup_logging()

#generate.my_run(instruments)
datasets_raw = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]
datasets_augmented = [pd.read_pickle(os.path.join(METADATA_DIR_AUGMENTED_RAW, f'data_{instrument.lower()}.pkl')) for instrument in instruments_aug]

from src.data.preprocessing import get_max_shape
max_spectrogram_size = max(map(lambda df: get_max_shape(df), datasets_raw+datasets_augmented))

from src.data.preprocessing import uniform_shape
uniform = lambda df: uniform_shape(df, max_spectrogram_size)

datasets_raw = list(map(uniform,datasets_raw))
datasets_augmented = list(map(uniform,datasets_augmented))
datasets_augmented = list(map(lambda df: df[['spectrogram','class_ID', 'class_name','augmentation']],datasets_augmented))
datasets_augmented = list(map(lambda df: df.reset_index(drop=True), datasets_augmented))

import random
import keras
import os, glob
import logging
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.models import model_from_json
from keras import backend as K

from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN_nodropout
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

setup_logging()
logger = logging.getLogger('src.train')

train_datas = []
test_datas = []

for dataset in datasets_raw:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

for dataset in datasets_augmented:
    train_data, test_data = train_test_split(dataset, augmented=True, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

train_data = train_data.sample(frac=1, random_state=1).reset_index(drop=True)
test_data = test_data.sample(frac=1, random_state=1).reset_index(drop=True)

most_shape = get_most_shape(train_data)

logger.info(f"Number of train samples: {len(train_data)}")
logger.info(f"Number of test samples: {len(test_data)}")
# most_shape = get_most_shape(dataset)
#train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.65)

X_train, y_train = features_target_split(train_data)
X_test, y_test = features_target_split(test_data)

# Reshape for CNN input
X_train, X_test = reshape_feature_CNN(X_train, size=max_spectrogram_size), reshape_feature_CNN(X_test, size=max_spectrogram_size)

# Preserve y_test values
y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN_nodropout(most_shape)
logger.info(str(cnn))

cnn.train(X_train, y_train, X_test, y_test, epochs=25)
cnn.evaluate(X_train, y_train, X_test, y_test)

if tf.__version__ != '1.8.0':
    predict_x=cnn.model.predict(X_test)
    predictions = np.argmax(predict_x,axis=1)
else:    
    predictions = cnn.model.predict_classes(X_test)
conf_matrix=confusion_matrix(y_test_values, predictions, labels=range(10))
logger.info('Confusion Matrix for classes {}:\n{}'.format(CLASSES, conf_matrix))
#cnn.save_model(name="model_all_data_augment_1")

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
The most frequent shape is (128, 107)
src.train - INFO 

In [6]:
test_instruments = instruments + instruments_aug
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, max_spectrogram_size, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

22/22 [==============================] - 0s 7ms/step - loss: 0.7460 - accuracy: 0.7743 - precision: 0.8125 - recall: 0.7141 - fmeasure: 0.7562
Test score for instrument: Guitar
	Test loss: 0.7459526658058167
	Test accuracy: 0.7742857336997986
	Test precision: 0.8124560713768005
	Test recall: 0.7140827775001526
	Test f1-score: 0.7562323212623596
10/10 [==============================] - 0s 7ms/step - loss: 0.4504 - accuracy: 0.9492 - precision: 0.9528 - recall: 0.9500 - fmeasure: 0.9514
Test score for instrument: Accordion
	Test loss: 0.45035186409950256
	Test accuracy: 0.9492063522338867
	Test precision: 0.9528225660324097
	Test recall: 0.949999988079071
	Test f1-score: 0.951388955116272
10/10 [==============================] - 0s 6ms/step - loss: 0.2914 - accuracy: 0.9365 - precision: 0.9392 - recall: 0.9344 - fmeasure: 0.9368
Test score for instrument: Violin
	Test loss: 0.2913728356361389
	Test accuracy: 0.9365079402923584
	Test precision: 0.939213752746582
	Test recall: 0.9343749880

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar']
instruments_aug = ['Accordion', 'Violin', 'Piano']

# from setup_logging import setup_logging
# setup_logging()

#generate.my_run(instruments)
datasets_raw = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED_NORMALIZED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]
datasets_augmented = [pd.read_pickle(os.path.join(METADATA_DIR_AUGMENTED_RAW_NORMALIZED, f'data_{instrument.lower()}.pkl')) for instrument in instruments_aug]

from src.data.preprocessing import get_max_shape
max_spectrogram_size = max(map(lambda df: get_max_shape(df), datasets_raw+datasets_augmented))

from src.data.preprocessing import uniform_shape
uniform = lambda df: uniform_shape(df, max_spectrogram_size)

datasets_raw = list(map(uniform,datasets_raw))
datasets_augmented = list(map(uniform,datasets_augmented))
datasets_augmented = list(map(lambda df: df[['spectrogram','class_ID', 'class_name','augmentation']],datasets_augmented))
datasets_augmented = list(map(lambda df: df.reset_index(drop=True), datasets_augmented))

import random
import keras
import os, glob
import logging
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.models import model_from_json
from keras import backend as K

from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN_nodropout
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

setup_logging()
logger = logging.getLogger('src.train')

train_datas = []
test_datas = []

for dataset in datasets_raw:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

for dataset in datasets_augmented:
    train_data, test_data = train_test_split(dataset, augmented=True, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

train_data = train_data.sample(frac=1, random_state=1).reset_index(drop=True)
test_data = test_data.sample(frac=1, random_state=1).reset_index(drop=True)

most_shape = get_most_shape(train_data)

logger.info(f"Number of train samples: {len(train_data)}")
logger.info(f"Number of test samples: {len(test_data)}")
# most_shape = get_most_shape(dataset)
#train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.65)

X_train, y_train = features_target_split(train_data)
X_test, y_test = features_target_split(test_data)

# Reshape for CNN input
X_train, X_test = reshape_feature_CNN(X_train, size=max_spectrogram_size), reshape_feature_CNN(X_test, size=max_spectrogram_size)

# Preserve y_test values
y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN_nodropout(most_shape)
logger.info(str(cnn))

cnn.train(X_train, y_train, X_test, y_test, epochs=8)
cnn.evaluate(X_train, y_train, X_test, y_test)

if tf.__version__ != '1.8.0':
    predict_x=cnn.model.predict(X_test)
    predictions = np.argmax(predict_x,axis=1)
else:    
    predictions = cnn.model.predict_classes(X_test)
conf_matrix=confusion_matrix(y_test_values, predictions, labels=range(10))
logger.info('Confusion Matrix for classes {}:\n{}'.format(CLASSES, conf_matrix))
#cnn.save_model(name="model_all_data_augment_1")

2023-05-20 15:15:19.477892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 15:15:19.612286: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-20 15:15:20.213779: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-20 15:15:20.213853: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 62
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 62
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 62
src.processing - INFO - Train test split completed
The most frequent shape is (128, 213)
src.train - INFO - N

2023-05-20 15:15:23.065962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


Epoch 1/8


2023-05-20 15:15:26.486640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2023-05-20 15:15:27.283620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


153/153 [==============================] - 7s 17ms/step - loss: 2.1523 - accuracy: 0.1961 - precision: 0.1993 - recall: 0.0191 - fmeasure: 0.0338 - val_loss: 1.6489 - val_accuracy: 0.3939 - val_precision: 0.7770 - val_recall: 0.1226 - val_fmeasure: 0.2064
Epoch 2/8
153/153 [==============================] - 2s 13ms/step - loss: 1.0857 - accuracy: 0.6262 - precision: 0.8212 - recall: 0.4446 - fmeasure: 0.5622 - val_loss: 0.6115 - val_accuracy: 0.8160 - val_precision: 0.9019 - val_recall: 0.7059 - val_fmeasure: 0.7888
Epoch 3/8
153/153 [==============================] - 2s 13ms/step - loss: 0.5497 - accuracy: 0.8144 - precision: 0.8908 - recall: 0.7533 - fmeasure: 0.8141 - val_loss: 0.4582 - val_accuracy: 0.8555 - val_precision: 0.9009 - val_recall: 0.8133 - val_fmeasure: 0.8537
Epoch 4/8
153/153 [==============================] - 2s 13ms/step - loss: 0.3311 - accuracy: 0.8897 - precision: 0.9247 - recall: 0.8587 - fmeasure: 0.8898 - val_loss: 0.3984 - val_accuracy: 0.8758 - val_precisio

In [2]:
test_instruments = instruments + instruments_aug
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, max_spectrogram_size, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

22/22 [==============================] - 0s 10ms/step - loss: 0.3014 - accuracy: 0.9100 - precision: 0.9193 - recall: 0.9046 - fmeasure: 0.9118
Test score for instrument: Guitar
	Test loss: 0.3013662099838257
	Test accuracy: 0.9100000262260437
	Test precision: 0.919274091720581
	Test recall: 0.9046266078948975
	Test f1-score: 0.9117681384086609
2/2 [==============================] - 0s 54ms/step - loss: 0.0130 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - fmeasure: 1.0000
Test score for instrument: Accordion
	Test loss: 0.01303529180586338
	Test accuracy: 1.0
	Test precision: 1.0
	Test recall: 1.0
	Test f1-score: 1.0
2/2 [==============================] - 0s 10ms/step - loss: 0.0257 - accuracy: 0.9839 - precision: 0.9844 - recall: 0.9844 - fmeasure: 0.9844
Test score for instrument: Violin
	Test loss: 0.025706904008984566
	Test accuracy: 0.9838709831237793
	Test precision: 0.984375
	Test recall: 0.984375
	Test f1-score: 0.984375
2/2 [==============================] - 0s 9ms